In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder,LabelEncoder,RobustScaler,TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.impute import SimpleImputer

from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier,XGBRegressor

In [59]:
df=pd.read_csv(R'C:\Users\pavan\Desktop\data science\decision tree\ensemble methods\FLIGHT\Data_Train.csv')

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [61]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [62]:
df['Date_of_Journey']=pd.to_datetime(df['Date_of_Journey'])
df['Dep_Time']=pd.to_datetime(df['Dep_Time'])
df['Arrival_Time']=pd.to_datetime(df['Arrival_Time'])

C:\Users\pavan\AppData\Local\Temp\ipykernel_1880\1511355351.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Date_of_Journey']=pd.to_datetime(df['Date_of_Journey'])
C:\Users\pavan\AppData\Local\Temp\ipykernel_1880\1511355351.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dep_Time']=pd.to_datetime(df['Dep_Time'])
C:\Users\pavan\AppData\Local\Temp\ipykernel_1880\1511355351.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Arrival_Time']=pd.to_datetime(df['Arrival_Time'])


In [63]:
df['day_of_journey']=df['Date_of_Journey'].dt.day
df['month_of_journey']=df['Date_of_Journey'].dt.month
df['year_of_journey']=df['Date_of_Journey'].dt.year

In [64]:
df['departed_time_hr']=df['Dep_Time'].dt.hour
df['departed_time_min']=df['Dep_Time'].dt.minute

In [65]:
df['arrival_hr']=df['Arrival_Time'].dt.hour
df['arrival_min']=df['Arrival_Time'].dt.minute

In [66]:
df[['Duration_Hr','Duration_min']]=df['Duration'].str.extract (r'(?:(\d+)h)?\s*(?:(\d+)m)?').fillna(0)

In [67]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,day_of_journey,month_of_journey,year_of_journey,departed_time_hr,departed_time_min,arrival_hr,arrival_min,Duration_Hr,Duration_min
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR ? DEL,2026-01-19 22:20:00,2026-03-22 01:10:00,2h 50m,non-stop,No info,3897,24,3,2019,22,20,1,10,2,50
1,Air India,2019-05-01,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,2026-01-19 05:50:00,2026-01-19 13:15:00,7h 25m,2 stops,No info,7662,1,5,2019,5,50,13,15,7,25
2,Jet Airways,2019-06-09,Delhi,Cochin,DEL ? LKO ? BOM ? COK,2026-01-19 09:25:00,2026-06-10 04:25:00,19h,2 stops,No info,13882,9,6,2019,9,25,4,25,19,0
3,IndiGo,2019-05-12,Kolkata,Banglore,CCU ? NAG ? BLR,2026-01-19 18:05:00,2026-01-19 23:30:00,5h 25m,1 stop,No info,6218,12,5,2019,18,5,23,30,5,25
4,IndiGo,2019-03-01,Banglore,New Delhi,BLR ? NAG ? DEL,2026-01-19 16:50:00,2026-01-19 21:35:00,4h 45m,1 stop,No info,13302,1,3,2019,16,50,21,35,4,45


In [68]:
df=df.drop(columns=['Date_of_Journey','Dep_Time','Arrival_Time','Duration','Additional_Info'])

In [69]:
df.head()

,Airline,Source,Destination,Route,Total_Stops,Price,day_of_journey,month_of_journey,year_of_journey,departed_time_hr,departed_time_min,arrival_hr,arrival_min,Duration_Hr,Duration_min
0,IndiGo,Banglore,New Delhi,BLR ? DEL,non-stop,3897,24,3,2019,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,2 stops,7662,1,5,2019,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,DEL ? LKO ? BOM ? COK,2 stops,13882,9,6,2019,9,25,4,25,19,0
3,IndiGo,Kolkata,Banglore,CCU ? NAG ? BLR,1 stop,6218,12,5,2019,18,5,23,30,5,25
4,IndiGo,Banglore,New Delhi,BLR ? NAG ? DEL,1 stop,13302,1,3,2019,16,50,21,35,4,45


In [70]:
df=df[['Airline', 'Source', 'Destination', 'Route', 'Total_Stops', 
       'day_of_journey', 'month_of_journey', 'year_of_journey',
       'departed_time_hr', 'departed_time_min', 'arrival_hr', 'arrival_min',
       'Duration_Hr', 'Duration_min','Price']]

In [71]:
df['Route']=df['Route'].fillna(df['Route'].mode()[0])
df['Total_Stops']=df['Total_Stops'].fillna(df['Total_Stops'].mode()[0])

In [72]:
x=df.drop(columns='Price')
y=df['Price']

In [73]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=42)

In [74]:
num=x_train.select_dtypes(include='number').columns
cat=x_train.select_dtypes(exclude='number').columns

In [75]:
df['Route'].nunique()

128

In [76]:
num_pipleline=Pipeline(
    steps=[
        ('num_scaling',StandardScaler())
    ]
)

airline_route_pipeline=Pipeline(
    steps=[
        ('airline_route_encoding',TargetEncoder())
    ]
)

source_destination_totalRoute_pipeline=Pipeline(
    steps=[
        ('source_encoding',OrdinalEncoder())
    ]
)


In [77]:
preprocessing=ColumnTransformer(
    transformers=[
        ('num_pipleline',num_pipleline,num),
        ('airline_route',airline_route_pipeline,['Airline','Route']),
        ('source_destination_totalroute',source_destination_totalRoute_pipeline,['Source','Destination','Total_Stops'])
    ]
)

In [ ]:
main_pipeline=Pipeline(
    steps=[
        ("preprocessing",preprocessing),
        ('model',XGBRegressor())
    ]
)

In [79]:
gridsearch=GridSearchCV(
    estimator=main_pipeline,
    param_grid={
        'model__criterion':['squared_error', 'absolute_error'],
        'model__max_depth':[None,5],
        'model__min_samples_split':[2,4],
        'model__min_samples_leaf':[1,3]
    },
    cv=3,
    verbose=3,
    n_jobs=-1,
    scoring='neg_mean_absolute_error'
)

In [80]:
main_pipeline.fit(x_train,y_train)

c:\Users\pavan\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipleline',
                                                  Pipeline(steps=[('num_scaling',
                                                                   StandardScaler())]),
                                                  Index(['day_of_journey', 'month_of_journey', 'year_of_journey',
       'departed_time_hr', 'departed_time_min', 'arrival_hr', 'arrival_min'],
      dtype='object')),
                                                 ('airline_route',
                                                  Pipeline(steps=[('airline_route_encoding',
                                                                   TargetEncoder...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [81]:
main_pipeline.score(x_train,y_train)

0.8687480688095093

In [82]:
main_pipeline.score(x_test,y_test)

0.8487258553504944